In [3]:
import sys
sys.path.append("../")
import pandas as pd
import datetime as dt
from api.oanda_api import OandaApi
from dateutil import parser
import timeit
import plotly.graph_objects as go
import numpy as np
from technicals import trend
from technicals import zone
from technicals import pattern
from technicals import bottom
from technicals import candle
from charting import draw
from scipy.signal import argrelextrema
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [4]:
def detect_setup(
    df,
    strength_col='bullish_strength_score',
    strength_threshold=0.7,
    lookahead=25,
    proximity_pips=0.0030,
    rolling_window=40,
    breakout_threshold=20  # in pips
):
    df = df.copy().reset_index(drop=True)
    df['setup_stage'] = None

    # Step 1: Bottom detection
    df['is_bottom'] = (
        (df['mid_l'] == df['mid_l'].rolling(window=rolling_window).min()) &
        (df['in_downtrend'] == True)
    )
    df.loc[df['is_bottom'], 'setup_stage'] = 'bottom'

    # Step 2: Track active zone
    df['active_zone_low'] = None
    df['active_zone_high'] = None

    current_low = None
    current_high = None
    last_bottom_idx = None
    breakout_found = False
    reentry_found = False
    confirmation_found = False

    for i in range(len(df)):
        if df.at[i, 'is_bottom']:
            # New setup: reset everything
            current_low = df.at[i, 'mid_l']
            current_high = df.at[i, 'mid_h']
            last_bottom_idx = i
            breakout_found = False
            reentry_found = False
            confirmation_found = False

        df.at[i, 'active_zone_low'] = current_low
        df.at[i, 'active_zone_high'] = current_high

        # Skip if setup was invalidated
        if current_high is None:
            continue

        max_allowed_low = current_high + breakout_threshold / 10000.0

        # Invalidate if price goes too far from zone
        if breakout_found and df.at[i, 'mid_l'] > max_allowed_low:
            current_low = None
            current_high = None
            last_bottom_idx = None
            breakout_found = False
            reentry_found = False
            confirmation_found = False
            df.at[i, 'active_zone_low'] = None
            df.at[i, 'active_zone_high'] = None
            continue

        # Step 2: Breakout
        if not breakout_found and df.at[i, 'mid_l'] > current_high:
            df.at[i, 'setup_stage'] = 'breakout'
            breakout_found = True
            continue  # Reentry can't be on breakout candle

        # Step 3: Reentry
        if breakout_found and not reentry_found:
            if current_low <= df.at[i, 'mid_l'] <= current_high:
                df.at[i, 'setup_stage'] = 'reentry'
                reentry_found = True
                continue  # Confirmation comes after reentry

        # Step 4: Confirmation (with spacing from bottom)
        if breakout_found and reentry_found and not confirmation_found:
            if df.at[i, strength_col] > strength_threshold:
                if i - last_bottom_idx >= 6:  # ✅ Enforce minimum spacing from bottom
                    df.at[i, 'setup_stage'] = 'confirmation'
                    confirmation_found = True

    return df

def detect_bottom_reversal_setups(
    df,
    strength_col='bullish_strength_score',
    strength_threshold=0.7,
    lookahead=25,
    proximity_pips=0.0030,
    rolling_window=40,
    breakout_threshold=20  # in pips (e.g. 30 = 0.0030)
):
    """
    Detects bottom reversal setups using new lows that occur only during downtrends.

    A setup is:
    1. A bottom candle (lowest in a window, while in_downtrend is True)
    2. A breakout candle (low + high > zone high)
    3. A reentry candle (low reenters the zone but never breaks below zone low)
    4. A strong bullish candle (close near zone high and strength score high)

    The setup is invalidated if any candle after reentry has a low greater than
    breakout_threshold above the zone high.

    Returns:
        DataFrame with setup_stage column added.
    """
    df = df.copy().reset_index(drop=True)
    df['setup_stage'] = None

    # Step 1: Tag bottom candles
    
    df.loc[df['is_bottom'], 'setup_stage'] = 'bottom'
    bottom_indexes = df.index[df['is_bottom']].tolist()

    for i in bottom_indexes:
        # 🔥 Always reset the active zone on every new bottom
        active_zone_low = df.at[i, 'mid_l']
        active_zone_high = df.at[i, 'mid_h']
        active_bottom_idx = i

        # Step 2: Look for breakout
        breakout_idx = None
        for j in range(i + 1, len(df)):
            row_j = df.iloc[j]
            if row_j['mid_l'] < active_zone_low:
                break  # Setup invalidated by lower low
            if row_j['mid_l'] > active_zone_high and row_j['mid_h'] > active_zone_high:
                breakout_idx = j
                break

        if breakout_idx is None or breakout_idx - i > lookahead:
            continue

        # Step 3: Reentry
        reentry_idx = None
        for k in range(breakout_idx + 1, breakout_idx + lookahead):
            if k >= len(df):
                break
            row_k = df.iloc[k]
            if active_zone_low <= row_k['mid_l'] <= active_zone_high:
                reentry_idx = k
                break
            if row_k['mid_l'] < active_zone_low:
                break  # Invalidated by new low

        if reentry_idx is None:
            continue

        # Step 4: Confirmation + breakout distance check
        confirmation_found = False
        breakout_distance_limit = active_zone_high + breakout_threshold / 10000.0
        df['breakout_distance_limit'] = breakout_distance_limit
        df['active_zone_high'] = active_zone_high
        df['breakout_threshold'] = breakout_threshold / 10000.0
        for m in range(reentry_idx + 1, reentry_idx + lookahead):
            if m >= len(df):
                break
            row_m = df.iloc[m]

            if row_m['mid_l'] < active_zone_low:
                break  # Invalidated by new low
            if row_m['mid_l'] > breakout_distance_limit:
                break  # Price went too far — invalidated

            if row_m[strength_col] >= strength_threshold:
                if abs(row_m['mid_l'] - active_zone_high) <= proximity_pips:
                    df.at[breakout_idx, 'setup_stage'] = 'breakout'
                    df.at[reentry_idx, 'setup_stage'] = 'reentry'
                    df.at[m, 'setup_stage'] = 'confirmation'
                    confirmation_found = True
                    break

        # No need to manually reset active_* vars — loop resets on next bottom

    return df

def find_support_resistance(df, price_col='mid_c', high_col='mid_h', low_col='mid_l', window=3, clustering_threshold=0.0050):
    """
    Identifies support and resistance levels in candlestick data.
    
    Args:
        df (pd.DataFrame): Your OHLC dataframe.
        price_col (str): Column name for close/mid price.
        high_col (str): Column name for highs.
        low_col (str): Column name for lows.
        window (int): Lookback window to detect local highs/lows.
        clustering_threshold (float): Maximum distance between levels to consider them the same zone.

    Returns:
        Tuple[List[float], List[float]]: (support_levels, resistance_levels)
    """
    
    local_min_idx = argrelextrema(df[low_col].values, np.less_equal, order=window)[0]
    local_max_idx = argrelextrema(df[high_col].values, np.greater_equal, order=window)[0]

    raw_supports = df.iloc[local_min_idx][low_col].values
    raw_resistances = df.iloc[local_max_idx][high_col].values

    def cluster_levels(levels):
        clustered = []
        levels = sorted(levels)
        for level in levels:
            if not clustered:
                clustered.append([level])
            elif abs(level - np.mean(clustered[-1])) <= clustering_threshold:
                clustered[-1].append(level)
            else:
                clustered.append([level])
        return [round(np.mean(group), 5) for group in clustered if len(group) >= 2]  # Only return stronger levels

    support_levels = cluster_levels(raw_supports)
    resistance_levels = cluster_levels(raw_resistances)

    return support_levels, resistance_levels

def get_zones_for_price(price, support_levels, resistance_levels, num_of_zones=3, min_gap=0.0, min_width=0.0015):
    """
    Returns non-overlapping (support, resistance) zones where the support is above the given price,
    and there's at least `min_gap` space and `min_width` size.

    Args:
        price (float): Current price.
        support_levels (list of float): Detected support levels.
        resistance_levels (list of float): Detected resistance levels.
        num_of_zones (int): Number of zones to return.
        min_gap (float): Minimum gap between zones.
        min_width (float): Minimum acceptable width of a zone.

    Returns:
        List of tuples: [(support1, resistance1), (support2, resistance2), ...]
    """

    support_levels = sorted(support_levels)
    resistance_levels = sorted(resistance_levels)

    zones = []
    last_resistance = price

    sup_above = [s for s in support_levels if s > price]

    for support in sup_above:
        if support <= last_resistance + min_gap:
            continue

        possible_resistances = [r for r in resistance_levels if r > support]
        for resistance in possible_resistances:
            width = resistance - support
            if width >= min_width:
                zones.append((support, resistance))
                last_resistance = resistance
                break  # Move on to the next zone

        if len(zones) == num_of_zones:
            break

    return zones

def attach_zones_to_confirmations(
    df,
    window=3,
    clustering_threshold=0.0050,
    num_of_zones=3
):
    """
    For each confirmation candle:
        - Attach relevant support/resistance zones based on past data only
        - Compute zone-to-stop-loss ratio using second zone
        - Add 'confirmation_zones', 'zone_sl_ratio', and 'meets_ratio' columns
    """
    from copy import deepcopy
    df['confirmation_zones'] = None
    df['zone_sl_ratio'] = None
    df['meets_ratio'] = False

    for i in range(len(df)):
        if df.at[i, 'setup_stage'] == 'confirmation':
            past_df = df.iloc[:i]
            if len(past_df) < window * 2:
                continue

            support_levels, resistance_levels = find_support_resistance(
                past_df,
                price_col='mid_c',
                high_col='mid_h',
                low_col='mid_l',
                window=window,
                clustering_threshold=clustering_threshold
            )

            current_price = df.at[i, 'mid_c']
            current_low = df.at[i, 'mid_l']

            zones = get_zones_for_price(
                price=current_price,
                support_levels=support_levels,
                resistance_levels=resistance_levels,
                num_of_zones=num_of_zones
            )

            df.at[i, 'confirmation_zones'] = deepcopy(zones)

            if len(zones) >= 2:
                zone_top = zones[1][1]  # Top of second zone (resistance)
                reward = zone_top - current_price
                risk = current_price - current_low

                if risk > 0:
                    ratio = reward / risk
                    df.at[i, 'zone_sl_ratio'] = round(ratio, 3)
                    df.at[i, 'meets_ratio'] = ratio >= 1.0

def simulate_trades(df, sl_pips=15, pair='EUR_USD'):
    pip_divisor = 100 if 'JPY' in pair else 10000
    pip_size = sl_pips / pip_divisor

    df['trade'] = None
    df['entry_price'] = None
    df['stop_loss'] = None
    df['take_profit'] = None
    df['pips'] = None

    active_trade = None

    for i in range(len(df)):
        row = df.iloc[i]

        # Check for new confirmation candle to enter trade
        if row['setup_stage'] == 'confirmation':
            zone = row.get('zone')
            if not isinstance(zone, tuple) or len(zone) != 2:
                continue

            zone_low = zone[0]
            entry_price = row['mid_c']
            stop_loss = zone_low - pip_size
            take_profit = entry_price + ((entry_price - stop_loss) * 2)

            active_trade = {
                'entry_idx': i,
                'entry_price': entry_price,
                'stop_loss': stop_loss,
                'take_profit': take_profit
            }

            df.at[df.index[i], 'trade'] = 'opened'
            df.at[df.index[i], 'entry_price'] = entry_price
            df.at[df.index[i], 'stop_loss'] = stop_loss
            df.at[df.index[i], 'take_profit'] = take_profit
            continue

        # If trade is active, check for SL/TP hit
        if active_trade:
            high = row['mid_h']
            low = row['mid_l']
            idx = df.index[i]

            if low <= active_trade['stop_loss']:
                df.at[idx, 'trade'] = 'closed - sl'
                df.at[idx, 'pips'] = round((active_trade['stop_loss'] - active_trade['entry_price']) / (1 / pip_divisor), 1)
                active_trade = None
            elif high >= active_trade['take_profit']:
                df.at[idx, 'trade'] = 'closed - tp'
                df.at[idx, 'pips'] = round((active_trade['take_profit'] - active_trade['entry_price']) / (1 / pip_divisor), 1)
                active_trade = None

def apply_technicals(df):
    df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
    trend.apply_downtrend(df)
    bottom.apply_bottom_zones(df)
    zone.apply_zone_exits_and_reentries(df, 50, 'EUR_USD')
    candle.detect_strong_bullish(df)
    #need to make this not return a df but add in place
    df = detect_setup(df)
    attach_zones_to_confirmations(df)
    return df

def plot_trades(df):
    fig = go.Figure()

    x_start = df['sTime'].iloc[0]
    x_end = df['sTime'].iloc[-1]
    y_min = df['mid_l'].min()
    y_max = df['mid_h'].max()
    padding = (y_max - y_min) * 0.1
    
    fig.add_trace(go.Candlestick(
        x=df.sTime,
        open=df.mid_o,
        high=df.mid_h,
        low=df.mid_l,
        close=df.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B', 
        decreasing_fillcolor='#CC2E3C',
        increasing_line_color='#24A06B',
        decreasing_line_color='#FF3A4C'
    ))

    df_trade_entries = df[df['trade'] == 'opened']

    fig.add_trace(go.Candlestick(
        x=df_trade_entries.sTime,
        open=df_trade_entries.mid_o,
        high=df_trade_entries.mid_h,
        low=df_trade_entries.mid_l,
        close=df_trade_entries.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#2d63ff', 
        decreasing_fillcolor='#2d63ff',
        increasing_line_color='#2d63ff',
        decreasing_line_color='#2d63ff'
    ))

    df_trade_closes = df[df['trade'].isin(['closed - tp', 'closed - sl'])]

    for i, open_row in df_trade_entries.iterrows():
        # Find the corresponding closing trade (assuming it's after the opened one)
        close_match = df_trade_closes[df_trade_closes['sTime'] > open_row['sTime']]
        if close_match.empty:
            continue  # No closing trade found

        close_row = close_match.iloc[0]  # Take the first closing one

        # Take Profit line
        fig.add_trace(go.Scatter(
            x=[open_row['sTime'], close_row['sTime']],
            y=[open_row['take_profit'], open_row['take_profit']],
            mode='lines',
            line=dict(color='green', width=1, dash='dot'),
            name='Take Profit',
            showlegend=False
        ))

        # Stop Loss line
        fig.add_trace(go.Scatter(
            x=[open_row['sTime'], close_row['sTime']],
            y=[open_row['stop_loss'], open_row['stop_loss']],
            mode='lines',
            line=dict(color='red', width=1, dash='dot'),
            name='Stop Loss',
            showlegend=False
        ))

    fig.update_yaxes(
        autorange=True,
        fixedrange=False,
        rangemode='normal',
        scaleratio=1,
        automargin=True,
        gridcolor="#1f292f"
    )

    fig.update_xaxes(
        gridcolor="#1f292f",
        rangeslider=dict(visible=True),
        nticks=5
    )

    fig.update_layout(
        width=1500,
        height=600,
        margin=dict(l=10,r=10,b=10,t=10),
        paper_bgcolor="#2c303c",
        plot_bgcolor="#2c303c",
        font=dict(size=10, color="#e1e1e1"),
        yaxis=dict(
            range=[y_min - padding, y_max + padding],
            fixedrange=False  # Allow zooming
        )
    )

    fig.show()
        

def filter_df_by_date(df, start, end):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    return df[(df['time'] >= start) & (df['time'] <= end)]

def summarize_trades(df, starting_balance=10000, risk_per_trade=0.02):

    # Filter only closed trades
    closed_trades = df[df['trade'].isin(['closed - sl', 'closed - tp'])].copy()

    if closed_trades.empty:
        print("No closed trades found.")
        return

    # Calculate metrics
    total_trades = len(closed_trades)
    total_pips = closed_trades['pips'].sum()
    win_rate = (closed_trades['trade'] == 'closed - tp').mean() * 100

    # Cumulative profit over time (simulate compounding returns with 2% risk)
    balance = starting_balance
    balances = [balance]

    for i, row in closed_trades.iterrows():
        risk_amount = balance * risk_per_trade
        if row['trade'] == 'closed - sl':
            balance -= risk_amount
        elif row['trade'] == 'closed - tp':
            reward = risk_amount * (row['pips'] / abs(row['pips']) if row['pips'] != 0 else 1)
            balance += reward
        balances.append(balance)

    closed_trades['cumulative_pips'] = closed_trades['pips'].cumsum()
    closed_trades['cumulative_balance'] = balances[1:]
    closed_trades['time'] = pd.to_datetime(closed_trades['time'])

    total_dollar_gain = balances[-1] - starting_balance
    percent_gain = (balances[-1] / starting_balance - 1) * 100

    # Print summary
    print(f"📈 Total Trades: {total_trades}")
    print(f"💰 Total Pips: {round(total_pips, 1)}")
    print(f"✅ Win Rate: {round(win_rate, 2)}%")
    print(f"💵 Total $ Gain: ${round(total_dollar_gain, 2)}")
    print(f"📊 Percent Gain: {round(percent_gain, 2)}%")

    # Plot Pips and Balance
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=closed_trades['time'],
        y=closed_trades['cumulative_pips'],
        mode='lines+markers',
        name='Cumulative Pips',
        line=dict(width=2),
        marker=dict(size=4)
    ))

    fig.add_trace(go.Scatter(
        x=closed_trades['time'],
        y=closed_trades['cumulative_balance'],
        mode='lines+markers',
        name='Account Balance ($)',
        line=dict(width=2),
        marker=dict(size=4),
        yaxis="y2"
    ))

    fig.update_layout(
        title="📈 Strategy Performance",
        xaxis_title="Time",
        yaxis=dict(title="Cumulative Pips"),
        yaxis2=dict(title="Account Balance ($)", overlaying='y', side='right'),
        template="plotly_white",
        hovermode="x unified",
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()

In [71]:
df = pd.read_pickle('../data/EUR_USD_M5.pkl')
# df = filter_df_by_date(df, "2024-06-01T00:00:00Z", "2025-01-01T00:00:00Z")

df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
trend.apply_downtrend(df)
bottom.apply_bottom_zones(df)
# zone.apply_zone_exits_and_reentries(df, 50, 'EUR_USD')
# candle.detect_strong_bullish(df)
# simulate_trades(df)
# df = apply_technicals(df)
# df = simulate_trades(df)
# df_between.head(45)

In [74]:
print("Total: ", (len(df)))
print("In Downtrend:", (df['in_downtrend'] == True).sum())
print("Bottoms: ", (df['is_bottom'] == True).sum())

Total:  744269
In Downtrend: 370995
Bottoms:  39363


In [ ]:
fig = draw.draw_candlestick_chart(df)


# draw.highlight_downtrend_candles(fig, df)
# draw.highlight_bottom_zones(fig, df)
# draw.highlight_exits_and_reentries(fig, df)
# draw.highlight_strong_bullish_candles(fig, df)

fig.show()

In [ ]:
def extract_trades(df):
    trades = []
    df.reset_index(drop=True, inplace=True)

    close_locs = df.index[df['trade'].isin(['closed - tp', 'closed - sl'])].tolist()
    print(f"Extracting {len(close_locs)} trade entries...")

    for close_loc in close_locs:
        open_index = close_loc - int(df.loc[close_loc, 'rows_since_entry'])
        bottom_index = open_index - int(df.loc[open_index, 'rows_since_bottom'])
        close_index = close_loc

        sliced_df = df.loc[bottom_index : close_index]
        trades.append(sliced_df)

    print(f"✅ Extracted {len(trades)} trade(s)")
    return trades

In [4]:
df = pd.read_pickle('../data/EUR_USD_M5.pkl')
print(len(df))

744418


In [5]:
df = pd.read_pickle('../backtesting/results/EUR_USD_H1_analyzed.pkl')
df = filter_df_by_date(df, "2024-09-03T17:00:00Z", "2024-09-11T00:00:00Z")
print("Opened trades:", (df['trade'] == 'opened').sum())
print(len(df))



Opened trades: 1
128


In [94]:
plot_trades(df)

In [6]:

print(len(df))
df.head(50)

128


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,sTime,ma_10,ma_50,ma_100,ma_150,in_downtrend,is_bottom,body,total_range,upper_wick,lower_wick,wick_ratio,avg_range,range_ok,wick_ok,close_near_high,strong_bullish,trade,entry_price,stop_loss,take_profit,pips,stage,rows_since_bottom,bottom_low,rows_since_entry
29121,2024-09-03 17:00:00+00:00,3909.0,1.10394,1.10402,1.10264,1.10280,1.10386,1.10394,1.10256,1.10272,1.10401,1.10409,1.10272,1.10288,s24-09-03 17:00,1.104222,1.105660,1.107620,1.110482,True,True,-0.00114,0.00138,0.00008,0.00016,-0.826087,0.001094,False,False,False,False,None,None,None,None,None,bottom,NaN,NaN,NaN
29122,2024-09-03 18:00:00+00:00,3456.0,1.10280,1.10393,1.10266,1.10380,1.10273,1.10386,1.10258,1.10373,1.10288,1.10400,1.10273,1.10388,s24-09-03 18:00,1.104116,1.105637,1.107527,1.110400,True,False,0.00100,0.00127,0.00013,0.00014,0.787402,0.001126,False,True,True,False,None,None,None,None,None,bottom,NaN,NaN,NaN
29123,2024-09-03 19:00:00+00:00,3816.0,1.10379,1.10422,1.10374,1.10418,1.10371,1.10414,1.10366,1.10410,1.10387,1.10430,1.10381,1.10425,s24-09-03 19:00,1.104132,1.105615,1.107443,1.110321,True,False,0.00039,0.00048,0.00004,0.00005,0.812500,0.001132,False,True,True,False,None,None,None,None,None,bottom,NaN,NaN,NaN
29124,2024-09-03 20:00:00+00:00,1448.0,1.10420,1.10446,1.10397,1.10433,1.10412,1.10438,1.10389,1.10426,1.10427,1.10453,1.10405,1.10440,s24-09-03 20:00,1.104207,1.105598,1.107371,1.110232,True,False,0.00013,0.00049,0.00013,0.00023,0.265306,0.001108,False,False,False,False,None,None,None,None,None,bottom,NaN,NaN,NaN
29125,2024-09-03 21:00:00+00:00,1116.0,1.10436,1.10462,1.10396,1.10460,1.10420,1.10447,1.10370,1.10445,1.10452,1.10479,1.10410,1.10474,s24-09-03 21:00,1.104304,1.105584,1.107311,1.110154,True,False,0.00024,0.00066,0.00002,0.00040,0.363636,0.001099,False,False,True,False,None,None,None,None,None,bottom,NaN,NaN,NaN
29126,2024-09-03 22:00:00+00:00,1027.0,1.10462,1.10469,1.10396,1.10458,1.10445,1.10461,1.10385,1.10449,1.10478,1.10478,1.10407,1.10466,s24-09-03 22:00,1.104282,1.105582,1.107241,1.110070,True,False,-0.00004,0.00073,0.00007,0.00062,-0.054795,0.001099,False,False,True,False,None,None,None,None,None,bottom,NaN,NaN,NaN
29127,2024-09-03 23:00:00+00:00,1862.0,1.10458,1.10506,1.10451,1.10486,1.10450,1.10499,1.10444,1.10478,1.10466,1.10514,1.10458,1.10493,s24-09-03 23:00,1.104154,1.105581,1.107176,1.109989,True,False,0.00028,0.00055,0.00020,0.00007,0.509091,0.001095,False,False,False,False,None,None,None,None,None,exit,NaN,NaN,NaN
29128,2024-09-04 00:00:00+00:00,4896.0,1.10482,1.10514,1.10466,1.10483,1.10474,1.10506,1.10457,1.10475,1.10489,1.10523,1.10473,1.10491,s24-09-04 00:00,1.104214,1.105586,1.107104,1.109912,True,False,0.00001,0.00048,0.00031,0.00016,0.020833,0.001095,False,False,False,False,None,None,None,None,None,exit,NaN,NaN,NaN
29129,2024-09-04 01:00:00+00:00,3479.0,1.10482,1.10564,1.10476,1.10532,1.10475,1.10556,1.10467,1.10525,1.10490,1.10572,1.10484,1.10539,s24-09-04 01:00,1.104322,1.105603,1.107037,1.109841,True,False,0.00050,0.00088,0.00032,0.00006,0.568182,0.001110,False,False,False,False,None,None,None,None,None,exit,NaN,NaN,NaN
29130,2024-09-04 02:00:00+00:00,3376.0,1.10534,1.10564,1.10495,1.10526,1.10527,1.10556,1.10487,1.10519,1.10541,1.10572,1.10503,1.10533,s24-09-04 02:00,1.104456,1.105601,1.106966,1.109769,True,False,-0.00008,0.00069,0.00030,0.00031,-0.115942,0.001092,False,False,False,False,None,None,None,None,None,exit,NaN,NaN,NaN


In [ ]:

df.reset_index(drop=True, inplace=True)
df[df['trade'] == 'opened'].head(20)
# df[df['trade'] == 'closed - tp'].head()
# df[df['trade'] == 'closed - sl'].head()
# df.head(30)
# trade_locs = df.index[df['trade'] == 'opened'].tolist()
# print(trade_locs[3])
# bottom_index = 1263 - int(df.loc[1263, 'rows_since_bottom'])
# print(bottom_index)
# df.iloc[bottom_index:1266].head(30)
# 12585 - 12418 opened
# 12855 - 12418 closed
# df[df['setup_stage'] == 'exit'].head(50)
# print(len(df_trades[1]))
# print((df_trades[0]['trade'] == 'opened').sum())

In [95]:
def draw_candlestick_chart(df):
    fig = go.Figure()

    trade_colors = {
        'opened': 'dodgerblue',
        'closed - tp': 'limegreen',
        'closed - sl': 'red'
    }

    fig.add_trace(go.Candlestick(
        x=df.sTime,
        open=df.mid_o,
        high=df.mid_h,
        low=df.mid_l,
        close=df.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B', 
        decreasing_fillcolor='#CC2E3C',
        increasing_line_color='#24A06B',
        decreasing_line_color='#FF3A4C'
    ))

    for trade_type, color in trade_colors.items():
        subset = df[df['trade'] == trade_type]
        fig.add_trace(go.Scatter(
            x=subset['sTime'],
            y=subset['mid_h'] + 0.0003,  # Slightly above the candle high
            mode='markers',
            marker=dict(color=color, size=8, symbol='circle'),
            name=trade_type,
            hoverinfo='text',
            text=[f"{trade_type} @ {p:.5f}" for p in subset['mid_c']]
        ))

    apply_layout(df, fig)

    return fig

def apply_layout(df, fig):
    x_start = df['sTime'].iloc[0]
    x_end = df['sTime'].iloc[-1]
    y_min = df['mid_l'].min()
    y_max = df['mid_h'].max()
    padding = (y_max - y_min) * 0.1

    fig.update_yaxes(
        autorange=True,
        fixedrange=False,
        rangemode='normal',
        scaleratio=0.5,
        automargin=True,
        gridcolor="#1f292f"
    )

    fig.update_xaxes(
        gridcolor="#1f292f",
        # rangeslider=dict(visible=True),
        nticks=5
    )

    fig.update_layout(
        width=1500,
        height=600,
        margin=dict(l=10,r=10,b=10,t=10),
        paper_bgcolor="#2c303c",
        plot_bgcolor="#2c303c",
        font=dict(size=10, color="#e1e1e1")
    )

    fig.update_layout(
        yaxis=dict(
            range=[y_min - padding, y_max + padding],
            fixedrange=False  # Allow zooming
        )
    )

In [ ]:


fig = draw_candlestick_chart(df)

draw.highlight_bottom_zones(fig, df, 'yellow')
draw.highlight_strong_bullish_candles(fig, df, 'blue')

fig.show()



In [58]:
df = pd.read_pickle('../backtesting/results/EUR_USD_M5_analyzed.pkl')
print("opened trades:", (df['trade'] == 'opened').sum())
print("closed trades:", df['trade'].isin(['closed - tp', 'closed - sl']).sum())

opened trades: 107
closed trades: 106


In [78]:
pairs = ["EUR_USD"]
granularities = ['M5']

for pair in pairs:
    for g in granularities:
        print(f"{pair} {g}")
        df = pd.read_pickle(f"../backtesting/results/{pair}_{g}_analyzed.pkl")
        summarize_trades(df)

EUR_USD M5
📈 Total Trades: 53
💰 Total Pips: -6.3
✅ Win Rate: 52.83%
💵 Total $ Gain: $506.47
📊 Percent Gain: 5.06%


In [95]:
pairs = ["EUR_USD"]
granularities = ['M5', 'M30', 'H1', 'H4']

for pair in pairs:
    for g in granularities:
        print(f"{pair} {g}")
        df = pd.read_pickle(f"../backtesting/results/{pair}_{g}_analyzed.pkl")
        summarize_trades(df)

EUR_USD M5
📈 Total Trades: 332
💰 Total Pips: 150.9
✅ Win Rate: 65.66%
💵 Total $ Gain: $64922.12
📊 Percent Gain: 649.22%


EUR_USD M30
📈 Total Trades: 68
💰 Total Pips: -189.9
✅ Win Rate: 61.76%
💵 Total $ Gain: $3585.8
📊 Percent Gain: 35.86%


EUR_USD H1
📈 Total Trades: 18
💰 Total Pips: -9.8
✅ Win Rate: 66.67%
💵 Total $ Gain: $1234.62
📊 Percent Gain: 12.35%


EUR_USD H4
📈 Total Trades: 7
💰 Total Pips: 30.6
✅ Win Rate: 71.43%
💵 Total $ Gain: $603.59
📊 Percent Gain: 6.04%


In [79]:
pairs = ["AUD_USD", "EUR_USD", "GBP_USD", "USD_CHF", "USD_JPY"]
granularities = ['M5', 'M30', 'H1', 'H4']

for pair in pairs:
    for g in granularities:
        print(f"{pair} {g}")
        df = pd.read_pickle(f"../backtesting/results/{pair}_{g}_analyzed.pkl")
        summarize_trades(df)

AUD_USD M5


FileNotFoundError: [Errno 2] No such file or directory: '../backtesting/results/AUD_USD_M5_analyzed.pkl'

In [ ]:
pairs = ["AUD_USD", "EUR_USD", "GBP_USD", "USD_CHF", "USD_JPY", "NZD_USD", "USD_CAD"]
for pair in pairs:
    print(pair)
    summarize_trades(f"../backtesting/results/{pair}_H1_analyzed.pkl")

In [ ]:
df_an = pd.read_pickle('../data/GBP_USD_H4.pkl').copy()
def apply_technicals(df):
    df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
    trend.apply_downtrend(df)
    df['bullish_strength_score'] = df.apply(pattern.bullish_strength, axis=1)
    #need to make this not return a df but add in place
    df = pattern.detect_bottom_reversal_setups(df)
    zone.attach_zones_to_confirmations(df)
    return df

df_an = apply_technicals(df_an)
df_an.head(40)